In [0]:
%sql
  SELECT
    t2.job_id,
    t2.result_state,
    "https://experian-serasa-ecs-labs.cloud.databricks.com/jobs/"||job_id as link,
    ROW_NUMBER() OVER(PARTITION BY t1.workspace_id, t2.job_id ORDER BY usage_start_time DESC) as rn
  FROM system_prd.billing.usage t1
    INNER JOIN system_prd.lakeflow.job_run_timeline t2
      ON
        t1.workspace_id=t2.workspace_id
        AND t1.usage_metadata.job_id = t2.job_id
        AND t1.usage_metadata.job_run_id = t2.run_id
  WHERE
    t1.sku_name LIKE '%JOBS%' AND 
    t1.usage_date >= CURRENT_DATE()
    AND t1.workspace_id = dataops_prd.libraries.get_workspace_id()
    AND result_state IN ('ERROR', 'FAILED', 'TIMED_OUT')
  QUALIFY rn = 1